# Loading Responses

In [1]:
options(warn=1)
if(!require('lme4')) {
    install.packages('lme4', repos='https://cloud.r-project.org')
    library('lme4')
}
if(!require('readr')) {
    install.packages('readr', repos='https://cloud.r-project.org')
    library('readr')
}
if(!require('marginaleffects')) {
    install.packages('marginaleffects', repos='https://cloud.r-project.org')
    library('marginaleffects')
}
if(!require('emmeans')) {
    install.packages('emmeans', repos='https://cloud.r-project.org')
    library('emmeans')
}
if(!require('car')) {
    install.packages('car', repos='https://cloud.r-project.org')
    library('car')
}
df <- read_csv("gaze-analysis/result_1743457603_20250506_20250506F.csv", na = "empty", col_select = c("Accuracy", "Group", "GroupKind", "Angle", "Proximity", "n_candidates", "Actor", "Candidates", "Stimulus_ID", "Prompt_ID", "Participant_ID", "list_id", "Run_ID", "Part"), col_types = cols(
    Accuracy = col_logical(),
    Group = col_factor(),
    GroupKind = col_factor(),
    Angle = col_factor(),
    Proximity = col_integer(),
    n_candidates = col_integer(),
    Actor = col_factor(c('X', 'Y')),
    Candidates = col_factor(),
    Stimulus_ID = col_factor(),
    Prompt_ID = col_factor(),
    Participant_ID = col_factor(),
    list_id = col_factor(),
    Run_ID = col_character(),
    Part = col_character(),
),show_col_types = TRUE)
df <- df[df$Group == "Humans" & df$list_id != -1 & df$Part != "p0", ]
df$offset <- log(1/df$n_candidates / (1 - 1/df$n_candidates))
df$Proximity <- df$Proximity - mean(df$Proximity)
df$n_candidates <- df$n_candidates - mean(df$n_candidates)

Loading required package: lme4

Loading required package: Matrix

Loading required package: readr

Loading required package: marginaleffects

Loading required package: emmeans

Welcome to emmeans.
Caution: You lose important information if you filter this package's results.
See '? untidy'

Loading required package: car

Loading required package: carData

Rows: 156780 Columns: 14
── Column specification ────────────────────────────────────────────────────────────────
Delimiter: ","
chr (2): Run_ID, Part
int (2): Proximity, n_candidates
lgl (1): Accuracy
fct (9): Stimulus_ID, Prompt_ID, Participant_ID, Group, GroupKind, Angle, Ac...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


# Fitting models

In [2]:
model0 <- glmer(Accuracy ~ Angle + Proximity + n_candidates + Actor + (1|Actor:Candidates) + (1|Stimulus_ID) + (1|Participant_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“unable to evaluate scaled gradient”
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge: degenerate  Hessian with 2 negative eigenvalues”


In [3]:
model1 <- glmer(Accuracy ~ Angle + Proximity + n_candidates + Actor + (1|Stimulus_ID) + (1|Participant_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

In [5]:
model1

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Accuracy ~ Angle + Proximity + n_candidates + Actor + (1 | Stimulus_ID) +  
    (1 | Participant_ID)
   Data: df
 Offset: offset
      AIC       BIC    logLik  deviance  df.resid 
1289.9295 1337.6539 -636.9648 1273.9295      2872 
Random effects:
 Groups         Name        Std.Dev.
 Stimulus_ID    (Intercept) 1.6691  
 Participant_ID (Intercept) 0.6123  
Number of obs: 2880, groups:  Stimulus_ID, 900; Participant_ID, 64
Fixed Effects:
 (Intercept)     Angleleft    Angleright     Proximity  n_candidates  
      7.3629       -3.1946       -3.1319       -1.2947       -0.6510  
      ActorY  
     -0.9936  

In [4]:
model2 <- glmer(Accuracy ~ Angle + Proximity + n_candidates + Actor + (1|Stimulus_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

# Model selection

In [6]:
anova(model1, model2)

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
model2,7,1302.104,1343.862,-644.0518,1288.104,NA,NA,NA
model1,8,1289.930,1337.654,-636.9648,1273.930,14.17404,1,0.0001666543


In [7]:
model <- model1

# Interpretation

In [8]:
summary(model)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Accuracy ~ Angle + Proximity + n_candidates + Actor + (1 | Stimulus_ID) +  
    (1 | Participant_ID)
   Data: df
 Offset: offset

     AIC      BIC   logLik deviance df.resid 
  1289.9   1337.7   -637.0   1273.9     2872 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-7.4876  0.0368  0.0893  0.1927  1.9914 

Random effects:
 Groups         Name        Variance Std.Dev.
 Stimulus_ID    (Intercept) 2.7860   1.6691  
 Participant_ID (Intercept) 0.3749   0.6123  
Number of obs: 2880, groups:  Stimulus_ID, 900; Participant_ID, 64

Fixed effects:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)    7.3629     0.5270  13.971  < 2e-16 ***
Angleleft     -3.1946     0.4221  -7.568 3.78e-14 ***
Angleright    -3.1319     0.4237  -7.391 1.46e-13 ***
Proximity     -1.2947     0.1682  -7.695 1.41e-14 ***
n_candidates  -0.6510     0.1527  -

In [9]:
log_odds_means <- emmeans(model, spec = "Angle")
print(log_odds_means)
pairs(log_odds_means)
log_odds_means <- emmeans(model, spec = "Angle", regrid = "response")
print(log_odds_means)
pairs(log_odds_means)
               
vc_list <- lapply(VarCorr(model), function(x) attr(x, "stddev")^2)
vc <- sum(unlist(vc_list))
effect_size_denom_t <- sqrt(vc + pi^2/3)

res <- summary(model)$coefficients
res <- as.data.frame(res)
res <- cbind(term = rownames(res), res)
rownames(res) <- NULL
res$effect_size_denom <- effect_size_denom_t
res$effect_size <- res$Estimate / effect_size_denom_t
res

 Angle emmean    SE  df asymp.LCL asymp.UCL
 front   6.52 0.494 Inf      5.55      7.49
 left    3.32 0.274 Inf      2.79      3.86
 right   3.39 0.277 Inf      2.85      3.93

Results are averaged over the levels of: Actor 
Results are given on the logit (not the response) scale. 
Confidence level used: 0.95 


 contrast      estimate    SE  df z.ratio p.value
 front - left    3.1946 0.422 Inf   7.568  <.0001
 front - right   3.1319 0.424 Inf   7.391  <.0001
 left - right   -0.0627 0.247 Inf  -0.253  0.9652

Results are averaged over the levels of: Actor 
Results are given on the log odds ratio (not the response) scale. 
P value adjustment: tukey method for comparing a family of 3 estimates 

 Angle   prob       SE  df asymp.LCL asymp.UCL
 front 0.9983 0.000806 Inf    0.9968    0.9999
 left  0.9614 0.010000 Inf    0.9417    0.9810
 right 0.9636 0.009520 Inf    0.9450    0.9823

Results are averaged over the levels of: Actor 
Confidence level used: 0.95 


 contrast      estimate      SE  df z.ratio p.value
 front - left   0.03698 0.00964 Inf   3.835  0.0004
 front - right  0.03474 0.00914 Inf   3.800  0.0004
 left - right  -0.00225 0.00887 Inf  -0.253  0.9653

Results are averaged over the levels of: Actor 
P value adjustment: tukey method for comparing a family of 3 estimates 

term,Estimate,Std. Error,z value,Pr(>|z|),effect_size_denom,effect_size
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),7.3628635,0.5270027,13.971206,2.336359e-44,2.539833,2.8989553
Angleleft,-3.1945815,0.4220948,-7.568399,3.778531e-14,2.539833,-1.2577917
Angleright,-3.1318812,0.4237471,-7.390921,1.458154e-13,2.539833,-1.2331050
Proximity,-1.2947018,0.1682499,-7.695112,1.413702e-14,2.539833,-0.5097586
n_candidates,-0.6509610,0.1527348,-4.262034,2.025750e-05,2.539833,-0.2563007
ActorY,-0.9935805,0.2397990,-4.143389,3.422100e-05,2.539833,-0.3911991
